In [1]:
import pandas as pd
import os
import json
import numpy as np
from IPython.display import Markdown, display
import ast
# Cho phép hiển thị toàn bộ nội dung transcript
# pd.set_option('display.max_colwidth', None)

# Explode cột category để mỗi

os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")

## Full dataset


In [2]:
full_df = pd.read_parquet('data/preprocessed_videos.parquet')
# full_df.info()

## Transcript dataset


In [3]:
transcript_df = pd.read_parquet('data/transcript_video_6_authors.parquet')
transcript_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   author.uniqueId            1335 non-null   object
 1   desc                       1335 non-null   object
 2   hashtags                   1335 non-null   object
 3   hashtag_count              1335 non-null   int64 
 4   video.id                   1335 non-null   object
 5   transcript                 1318 non-null   object
 6   takeaway_1                 1315 non-null   object
 7   takeaway_2                 1315 non-null   object
 8   takeaway_3                 1315 non-null   object
 9   transcript_call_to_action  1327 non-null   object
 10  transcript_curiosity_gap   1327 non-null   object
dtypes: int64(1), object(10)
memory usage: 114.9+ KB


## Đọc các file chứa phân tích transcript


In [4]:
# Đường dẫn folder chứa JSON
json_folder = 'data/transcript-6-users'

# Danh sách lưu kết quả
json_data_list = []

# Lặp qua từng ID trong DataFrame
for video_id in transcript_df['video.id']:
    file_path = os.path.join(json_folder, f"{video_id}.json")
    if not os.path.exists(file_path):
        print(f"⚠️ File không tồn tại: {file_path}")
        continue

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = json.load(f)
        # Thêm video.id vào nội dung JSON để merge được
        content['video.id'] = video_id
        json_data_list.append(content)
    except Exception as e:
        print(f"❌ Lỗi khi đọc file {file_path}: {e}")

# Chuyển thành DataFrame từ danh sách JSON
json_df = pd.DataFrame(json_data_list)
# json_df = json_df.merge(
#     transcript_df[['video.id', 'transcript']], on='video.id')
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   category            1335 non-null   object
 1   main_content_focus  1255 non-null   object
 2   structure_style     1255 non-null   object
 3   hook_type           1255 non-null   object
 4   tone_of_voice       1255 non-null   object
 5   pacing              1255 non-null   object
 6   has_cta             1255 non-null   object
 7   cta_type            1255 non-null   object
 8   emotion_keywords    1255 non-null   object
 9   hook_content        1255 non-null   object
 10  cta_phrases         1255 non-null   object
 11  highlight_phrases   1255 non-null   object
 12  notable_elements    1255 non-null   object
 13  gemini_analysis     1254 non-null   object
 14  video.id            1335 non-null   object
dtypes: object(15)
memory usage: 156.6+ KB


In [5]:
# json_df[json_df['category'] == "Không liên quan ẩm thực"]

## Tiền xử lý content features trích xuất từ transcript


In [6]:
json_df.columns

Index(['category', 'main_content_focus', 'structure_style', 'hook_type',
       'tone_of_voice', 'pacing', 'has_cta', 'cta_type', 'emotion_keywords',
       'hook_content', 'cta_phrases', 'highlight_phrases', 'notable_elements',
       'gemini_analysis', 'video.id'],
      dtype='object')

In [7]:
cleanned_json_df = json_df.copy()

### Hàm để tiền xử ly


In [8]:
valid_labels_dict = {
    "category": [
        "Review quán ăn",
        "Review sản phẩm ăn uống",
        "Mukbang",
        "Nấu ăn",
        "Không liên quan ẩm thực"
    ],
    "main_content_focus": [
        "Món ăn",
        "Quán ăn",
        "Sản phẩm ăn uống",
        "Công thức nấu ăn",
        "Cảm nhận trải nghiệm"
    ],
    "structure_style": [
        "Review mô tả",
        "Review trải nghiệm",
        "Kể chuyện",
        "Hướng dẫn",
        "So sánh",
        "Kịch hóa",
        "Phỏng vấn",
        "Chia sẻ thông tin"
    ],
    "hook_type": [
        "Gây tò mò",
        "Kể chuyện",
        "Vào thẳng vấn đề",
        "Đặt câu hỏi",
        "Phản hồi bình luận",
        "Khuyến mãi",
        "So sánh",
        "Giật tít",
        "Gây tranh cãi"
    ],
    "tone_of_voice": [
        "Hào hứng",
        "Hài hước",
        "Thân mật",
        "Ngọt ngào",
        "Ngạc nhiên",
        "Trung lập",
        "Nghiêm túc",
        "Khó chịu",
        "Giận dữ",
        "Bốc phốt"
    ],
    "pacing": [
        "Nhanh",
        "Chậm",
        "Thay đổi"
    ],
    "cta_type": [
        "Follow",
        "Like",
        "Share",
        "Comment",
        "Lưu lại",
        "Mời trải nghiệm",
        "Xem tiếp",
        "Liên hệ",
        "Khác"
    ]
}

In [9]:
def filter_valid_labels(df, column_name, valid_labels, default_label=None):
    """
    Lọc và chuẩn hóa cột chứa list nhãn:
    - Nếu default_label được chỉ định: mọi nhãn không hợp lệ sẽ được thay bằng default_label
    - Nếu default_label = None: chỉ giữ lại các nhãn hợp lệ, bỏ nhãn sai
    - Nếu sau khi lọc mà danh sách rỗng → giữ rỗng hoặc gán [default_label] nếu có

    Args:
        df (pd.DataFrame): DataFrame gốc
        column_name (str): Tên cột cần xử lý
        valid_labels (list): Danh sách các nhãn hợp lệ
        default_label (str or None): Nhãn thay thế cho nhãn không hợp lệ, hoặc gán nếu list rỗng

    Returns:
        pd.DataFrame: DataFrame đã xử lý
    """
    def clean_labels(x):
        if isinstance(x, list):
            if default_label is not None:
                # Thay mọi nhãn không hợp lệ bằng default_label
                return [label if label in valid_labels else default_label for label in x]
            else:
                # Giữ lại nhãn hợp lệ, bỏ nhãn sai
                return [label for label in x if label in valid_labels]
        return x  # Nếu không phải list → giữ nguyên

    df[column_name] = df[column_name].apply(clean_labels)
    return df

In [10]:
for field, valid_labels in valid_labels_dict.items():
    print(f"\n📊 Value counts for: {field}")

    # Flatten danh sách trong từng dòng thành 1 list duy nhất
    all_labels = cleanned_json_df[field].explode()

    # Tính value_counts và in ra
    print(all_labels.value_counts())


📊 Value counts for: category
category
Nấu ăn                     526
Review sản phẩm ăn uống    458
Review quán ăn             286
Không liên quan ẩm thực    108
Mukbang                     15
Review trải nghiệm          10
Kể chuyện                    3
Kịch hóa                     3
Làm đẹp                      2
Sinh hoạt                    1
Khuyến mãi                   1
So sánh                      1
Chia sẻ thông tin            1
Name: count, dtype: int64

📊 Value counts for: main_content_focus
main_content_focus
Cảm nhận trải nghiệm    880
Món ăn                  767
Sản phẩm ăn uống        426
Công thức nấu ăn        395
Quán ăn                 282
Name: count, dtype: int64

📊 Value counts for: structure_style
structure_style
Review trải nghiệm    885
Hướng dẫn             442
Kể chuyện             254
Review mô tả          112
Kịch hóa               14
Phỏng vấn              12
So sánh                10
Chia sẻ thông tin       7
Name: count, dtype: int64

📊 Value counts for:

In [11]:
# Nhãn mặc định nếu cần (tuỳ chỉnh theo từng trường nếu muốn)
default_label = "Khác"

# Xử lý tất cả các trường
for column_name, valid_labels in valid_labels_dict.items():
    cleanned_json_df = filter_valid_labels(
        cleanned_json_df, column_name, valid_labels, default_label=default_label)

In [12]:
for field, valid_labels in valid_labels_dict.items():
    print(f"\n📊 Value counts for: {field}")

    # Flatten danh sách trong từng dòng thành 1 list duy nhất
    all_labels = cleanned_json_df[field].explode()

    # Tính value_counts và in ra
    print(all_labels.value_counts())


📊 Value counts for: category
category
Nấu ăn                     526
Review sản phẩm ăn uống    458
Review quán ăn             286
Không liên quan ẩm thực    108
Khác                        22
Mukbang                     15
Name: count, dtype: int64

📊 Value counts for: main_content_focus
main_content_focus
Cảm nhận trải nghiệm    880
Món ăn                  767
Sản phẩm ăn uống        426
Công thức nấu ăn        395
Quán ăn                 282
Name: count, dtype: int64

📊 Value counts for: structure_style
structure_style
Review trải nghiệm    885
Hướng dẫn             442
Kể chuyện             254
Review mô tả          112
Kịch hóa               14
Phỏng vấn              12
So sánh                10
Chia sẻ thông tin       7
Name: count, dtype: int64

📊 Value counts for: hook_type
hook_type
Vào thẳng vấn đề      793
Gây tò mò             195
Kể chuyện             157
Đặt câu hỏi            92
Phản hồi bình luận     30
Khuyến mãi              9
Giật tít                4
So sánh       

In [13]:
# 1. Các cột cần kiểm tra thiếu dữ liệu (trừ 'category')
columns_to_check = [col for col in valid_labels_dict.keys()
                    if col != "category"]

# 2. Lọc dòng có category không chứa "Không liên quan ẩm thực"
mask_valid_category = cleanned_json_df['category'].apply(
    lambda x: isinstance(x, list) and "Không liên quan ẩm thực" not in x
)

# 3. Lọc dòng có ít nhất một cột khác bị None (hoặc NaN nếu có thể)
mask_has_none = cleanned_json_df[columns_to_check].isnull().any(axis=1)

# 4. Kết hợp 2 điều kiện
rows_with_issues = cleanned_json_df[mask_valid_category & mask_has_none]

# 5. Xem kết quả
rows_with_issues[['video.id'] + columns_to_check + ['category']]

,video.id,main_content_focus,structure_style,hook_type,tone_of_voice,pacing,cta_type,category


### Chuyển các dòng "Không liên quan ẩm thực" thành NaN


In [14]:
# Tạo mask cho các dòng có category chứa "Không liên quan ẩm thực"
mask_ko_lien_quan = cleanned_json_df['category'].apply(
    lambda x: isinstance(x, list) and "Không liên quan ẩm thực" in x)

# Lấy danh sách các cột cần gán NaN (trừ 'video.id' và 'category')
cols_to_null = [col for col in cleanned_json_df.columns if col not in [
    'video.id', 'category']]

# Gán NaN cho các dòng được chọn
cleanned_json_df.loc[mask_ko_lien_quan, cols_to_null] = np.nan

### Chuyển thành list và Đồng bộ các cột có list/string rỗng


In [15]:
def parse_list_column(value):
    # Nếu là None hoặc NaN hoặc list toàn NaN → trả về []
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return []
    if isinstance(value, list) and all(isinstance(v, float) and pd.isna(v) for v in value):
        return []

    try:
        parsed = ast.literal_eval(value) if isinstance(
            value, str) and value.startswith('[') else value
        if isinstance(parsed, list):
            return parsed
        else:
            return [parsed]
    except:
        return []


# def parse_list_column(value):
#     try:
#         # Nếu là string dạng list → ép kiểu
#         parsed = ast.literal_eval(value) if isinstance(
#             value, str) and value.startswith('[') else value
#         return parsed if isinstance(parsed, list) else [parsed]
#     except:
#         return [value]


def fill_none_string_column(value):
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return ""
    try:
        if pd.isna(value):
            return ""
    except:
        pass
    return value

In [16]:

# Các cột có dạng list (multi-label)
list_columns = ['category', 'hook_type', 'structure_style', 'tone_of_voice', 'emotion_keywords',
                'cta_type', 'main_content_focus', 'pacing', 'cta_phrases', 'highlight_phrases', 'notable_elements']

# Các cột dạng string
string_columns = ['hook_content', 'gemini_analysis']

# Áp dụng hàm xử lý list
for col in list_columns:
    if col in cleanned_json_df.columns:
        cleanned_json_df[col] = cleanned_json_df[col].apply(parse_list_column)
        cleanned_json_df[col] = cleanned_json_df[col].apply(lambda x: list(
            x) if isinstance(x, (list, np.ndarray, pd.Series)) else [])


# Áp dụng hàm xử lý string
for col in string_columns:
    if col in cleanned_json_df.columns:
        cleanned_json_df[col] = cleanned_json_df[col].apply(
            fill_none_string_column)

### Đổi tên `category` thành `categories` cho chuẩn.


In [17]:
cleanned_json_df = cleanned_json_df.rename(columns={'category': 'categories'})

### Merge với `transcript_df`


In [18]:
# Merge vào DataFrame gốc theo 'video.id'
merged_df = transcript_df.merge(cleanned_json_df, on='video.id', how='left')

In [19]:
merged_df.columns

Index(['author.uniqueId', 'desc', 'hashtags', 'hashtag_count', 'video.id',
       'transcript', 'takeaway_1', 'takeaway_2', 'takeaway_3',
       'transcript_call_to_action', 'transcript_curiosity_gap', 'categories',
       'main_content_focus', 'structure_style', 'hook_type', 'tone_of_voice',
       'pacing', 'has_cta', 'cta_type', 'emotion_keywords', 'hook_content',
       'cta_phrases', 'highlight_phrases', 'notable_elements',
       'gemini_analysis'],
      dtype='object')

In [20]:
merged_df = merged_df.drop(columns=['takeaway_1',
                                    'takeaway_2', 'takeaway_3', 'transcript_call_to_action',
                                    'transcript_curiosity_gap'])

## Lấy các cột thống kê


In [21]:
renamed_cols = {
    # 'author.uniqueId': 'username',
    'video.id': 'video.id',
    # "id": "id",
    "isAd": "isAd",
    "video.duration": "duration",
    "statsV2.playCount": "views",  # Lượt xem video
    "statsV2.diggCount": "likes",  # Lượt thích video
    "statsV2.commentCount": "comments",  # Lượt bình luận video
    "statsV2.shareCount": "shares",  # Lượt chia sẻ video
    'statsV2.collectCount': "collects",
    # "statsV2.playCount_rank": "views_rank",  # Lượt xem video
    # "statsV2.diggCount_rank": "likes_rank",  # Lượt thích video
    # "statsV2.commentCount_rank": "comments_rank",  # Lượt bình luận video
    # "statsV2.shareCount_rank": "shares_rank",  # Lượt chia sẻ video
    # 'statsV2.collectCount_rank': "collects_rank",
    'authorStats.diggCount': "user.likes",
    'authorStats.followerCount': "user.followers",
    'authorStats.followingCount': "user.followings",
    'authorStats.heartCount': "user.hearts",
    'authorStats.videoCount': "user.videos",
    'createTime': 'createTime'
}

In [22]:
stats_df = full_df[renamed_cols.keys()]

Dataframe cuối cùng để phân tích:


In [23]:
merged_df.columns

Index(['author.uniqueId', 'desc', 'hashtags', 'hashtag_count', 'video.id',
       'transcript', 'categories', 'main_content_focus', 'structure_style',
       'hook_type', 'tone_of_voice', 'pacing', 'has_cta', 'cta_type',
       'emotion_keywords', 'hook_content', 'cta_phrases', 'highlight_phrases',
       'notable_elements', 'gemini_analysis'],
      dtype='object')

In [24]:
# final_df = merged_df.merge(full_df, on=['author.uniqueId', 'video.id', 'video.duration'], how='left').rename(
#     columns=renamed_cols)

final_df = merged_df.merge(
    stats_df, on='video.id', how='left')

In [25]:
final_df.columns

Index(['author.uniqueId', 'desc', 'hashtags', 'hashtag_count', 'video.id',
       'transcript', 'categories', 'main_content_focus', 'structure_style',
       'hook_type', 'tone_of_voice', 'pacing', 'has_cta', 'cta_type',
       'emotion_keywords', 'hook_content', 'cta_phrases', 'highlight_phrases',
       'notable_elements', 'gemini_analysis', 'isAd', 'video.duration',
       'statsV2.playCount', 'statsV2.diggCount', 'statsV2.commentCount',
       'statsV2.shareCount', 'statsV2.collectCount', 'authorStats.diggCount',
       'authorStats.followerCount', 'authorStats.followingCount',
       'authorStats.heartCount', 'authorStats.videoCount', 'createTime'],
      dtype='object')

In [26]:
final_df.to_parquet(
    "data/content_features_6_users.parquet", index=False)